In [5]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
Alpha_Name = "AlphaNet_Original_Input_12.14"
path = '/home/wuwenjun/Data/' + Alpha_Name +'/'
target = "5d_ret"

In [6]:
# os.mkdir(path+"Pooling_Norm_Daily")
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 176745.39it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [8]:
""" Multi Processing"""
pool = mp.Pool(10)
t1 = time.time()
def test(file):
    array = pd.read_parquet(path + "Feat_Concat/" + file).set_index(["timestamp","ticker"])
    index = array.index
    array = np.array(array)
    array = torch.from_numpy(array.reshape(array.shape[0],-1,3))

    # Pooling
    max_pooling = torch.nn.MaxPool1d(kernel_size = 3, stride=1)(array) # Max Pooling
    avg_pooling = torch.nn.AvgPool1d(kernel_size = 3, stride=1)(array) # Mean Pooling
    min_pooling = -(torch.nn.MaxPool1d(kernel_size = 3, stride=1)(-array)) # MinPooling
    array = torch.cat([max_pooling,avg_pooling,min_pooling], axis=2)
    array = array.flatten(start_dim=1)
    array = pd.DataFrame(array.detach().numpy())
    array.index = index
    array.reset_index(inplace=True)
    # Daily Norm
    final = []
    for day in tqdm(array['timestamp'].drop_duplicates()):
        df = array[array['timestamp'] == day].set_index(['timestamp','ticker'])
        index = df.index
        # Batch
        df = torch.from_numpy(np.array(df)).to(torch.float) 
        batch = nn.BatchNorm1d(df.shape[1])
        df = batch(df)
        df = pd.DataFrame(df.detach().numpy())
        df.index = index
        final.append(df)

    final = pd.concat(final,axis=0)
    final.columns = [str(i) for i in range(final.shape[1])]
    final.reset_index(inplace=True)

    final.to_parquet(path + "Pooling_Norm_Daily/" + file)
    print("Save to : ", path + "Pooling_Norm_Daily/" + file)
    return None
for file in path_list:
#     test(file)
    pool.apply_async(test, args=(file,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 5/5 [00:00<00:00, 61.07it/s]


Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2015-01-01_2015-03-01.par


 45%|████▌     | 30/66 [00:05<00:06,  5.45it/s]

Save to : 

 59%|█████▉    | 38/64 [00:06<00:04,  5.47it/s]

...

/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2020-01-01_2020-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2018-03-01_2018-06-01.par


 27%|██▋       | 17/62 [00:00<00:00, 57.50it/s]

Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2018-06-01_2018-09-01.par


100%|██████████| 62/62 [00:01<00:00, 61.85it/s]


Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2019-03-01_2019-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2019-06-01_2019-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2020-03-01_2020-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2021-01-01_2021-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2018-09-01_2019-01-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2020-06-01_2020-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2019-09-01_2020-01-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2020-09-01_2021-01-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2021-03-01_2021-06-01.par
并行执行时间： 42


In [10]:
pd.set_option('display.max_columns', None)
pd.read_parquet("/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2015-01-01_2015-03-01.par")

,timestamp,ticker,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188
0,2015-02-16 09:30:00,1,0.207835,0.021603,-0.232302,0.102923,0.104148,0.236395,0.278850,-0.225329,-0.100755,0.259385,0.421858,0.381663,0.444228,0.814598,0.173874,0.571642,1.175144,1.073323,0.444840,0.815275,0.174882,0.444565,0.815076,0.175516,-0.248215,-0.279964,-0.047211,-0.297900,-0.999507,-1.038000,0.133856,0.223983,0.229365,0.505350,1.091140,1.245504,-0.589995,-0.407401,0.176291,0.506039,1.092819,1.246501,0.505738,1.092584,1.247309,0.029127,-0.119623,0.195165,0.142636,0.223226,0.337103,0.481127,0.325459,0.231147,-0.456628,0.138279,0.554142,0.482520,0.327595,0.231049,0.482168,0.326304,0.230653,0.114225,-0.168267,-0.433872,0.233194,-0.510114,-0.251352,0.816742,0.798614,0.119052,0.234074,-0.507536,-0.249942,0.233896,-0.510094,-0.251845,0.529395,0.698129,0.578325,-0.807911,-0.176545,0.453865,0.530590,0.700219,0.579743,0.530259,0.699260,0.579030,-1.783878,-1.379385,-0.789560,0.227190,0.327937,0.364581,0.227202,0.326820,0.362730,-1.781165,-1.378666,-0.788926,-1.784895,-1.381820,-0.790502,0.227200,0.325990,0.361133,0.244314,0.283043,0.413693,0.245476,0.274015,0.390765,0.203350,0.250616,0.370272,0.231255,0.246264,0.255954,0.231790,0.266179,0.404866,1.623192,1.870501,2.521021,0.301921,-0.086809,-0.238868,-0.592634,-0.586870,-0.324868,-0.569037,-0.552629,-0.141187,-0.264807,-0.243747,-0.175149,-0.267160,-0.245432,-0.177168,-0.264807,-0.243451,-0.175149,-0.270408,-0.247713,-0.179367,-0.263521,-0.240379,-0.168633,0.220825,0.490968,0.877159,-0.038199,-0.067795,-1.161185,0.223819,0.494465,0.881653,0.225341,0.495021,0.881904,-0.815820,-0.929328,-0.545952,-0.668898,-0.960868,-0.449766,-1.017549,-1.135066,-0.549489,-0.848561,-0.943469,-0.614387,-0.778471,-0.952729,-0.458424,-0.150269,-0.150425,-0.050158,-0.299700,-0.036359,0.249617,-0.049063,-0.051807,-0.035366,-0.050169,-0.052982,-0.037844
1,2015-02-16 09:30:00,2,-0.008425,0.686842,0.923933,0.032671,0.302827,0.353685,-0.078937,0.582956,0.785258,0.059245,0.479820,0.740284,0.633762,0.666464,-0.190145,0.082877,0.658991,0.716015,0.633893,0.663907,-0.196824,0.634039,0.662821,-0.197979,0.231052,0.574803,0.620721,0.075810,0.410627,0.426701,0.186548,0.355746,0.433584,0.384214,0.023905,-0.521522,-0.232829,0.089149,-0.231224,0.384982,0.020998,-0.531177,0.384642,0.018376,-0.535036,0.094713,0.273760,0.354074,0.244432,0.385729,0.483805,0.778020,0.260817,-0.321160,-0.276150,0.370434,0.620650,0.778888,0.259054,-0.329360,0.779255,0.256923,-0.331461,0.249710,0.302733,0.213305,0.670804,-0.086282,-0.396093,-0.316182,0.048106,0.159994,0.671138,-0.090386,-0.408804,0.671525,-0.093790,-0.413617,0.591612,0.075723,-0.377689,0.023809,0.350679,0.018272,0.592768,0.072884,-0.387845,0.592534,0.070210,-0.391467,0.096950,-0.865420,-1.170692,0.227190,0.327914,0.364529,0.227202,0.326787,0.362655,0.097425,-0.870091,-1.169711,0.096430,-0.873500,-1.171344,0.227200,0.325989,0.361131,0.615152,0.958852,1.345552,0.701523,0.952714,1.513892,0.556350,0.844346,1.257248,0.646635,0.891480,1.006592,0.651538,0.935476,1.410979,4.162855,4.644673,5.654445,0.922344,0.863064,0.124509,-0.190290,-0.065724,0.418306,-0.380133,-0.301098,0.164518,-0.264807,-0.243747,-0.175149,-0.267160,-0.245432,-0.177168,-0.264807,-0.243451,-0.175149,-0.270408,-0.247713,-0.179367,-0.263521,-0.240379,-0.168633,0.070268,0.043035,0.395784,-0.042480,-0.058868,-0.467706,0.068629,0.044270,0.399711,0.068741,0.043565,0.